In [1]:
from github import Github
from dotenv import load_dotenv
import os

load_dotenv()
api = Github(os.getenv('GITHUB_TOKEN'))


/Users/nicolegros/Documents/school/release-eng/replication-project/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
import json
from github.Repository import Repository

org = "mirantis"

paginated_list = api.search_repositories(query="user:{}".format(org))
repo: Repository = paginated_list[0]

filename = "../data/processed/mirantis/repos.json"
repos = []
for repo in paginated_list:
    repos.append(repo.name)
with open(filename, "w") as f:
    print(f"        Saving to {filename}")
    json.dump(repos, f, indent=4)



        Saving to ../data/processed/mirantis/repos.json


In [41]:
from github.PaginatedList import PaginatedList
from github.Commit import Commit

processed = {}
commits: PaginatedList[Commit] = repo.get_commits()[:2]
for c in commits:
    processed[c.sha] = {
        "files": list(map(lambda file: file.filename, c.files)),
        "date": c.commit.committer.date
    }

In [46]:
import pandas as pd

df = pd.DataFrame.from_dict(processed, orient="index")
df.to_pickle(f"../data/processed/mirantis/{repo.name}.commits.pickle")

In [47]:
dfpick = pd.read_pickle(f"../data/processed/{org}/{repo.name}.commits.pickle")
dfpick

,files,date
9f487a598c226e4cd522b945e1b70af9184fd7f6,"[go.mod, go.sum, vendor/github.com/golang/prot...",2023-10-20 20:56:19+00:00
a4152906fcf5ee528d596fcadb9a118019f2f792,"[Makefile, docs/content/about/motivation.md, d...",2023-10-20 15:39:52+00:00


In [1]:
from src.repoextractor import RepoExtractor

RepoExtractor("../data/processed").extract_repo_names("mirantis")

/Users/nicolegros/Documents/school/release-eng/replication-project/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Saving to ../data/processed/mirantis/repos.json


[Repository(full_name="Mirantis/cri-dockerd"),
 Repository(full_name="Mirantis/virtlet"),
 Repository(full_name="Mirantis/k8s-AppController"),
 Repository(full_name="Mirantis/kqueen"),
 Repository(full_name="Mirantis/pipeline-library"),
 Repository(full_name="Mirantis/k8s-externalipcontroller"),
 Repository(full_name="Mirantis/compliance"),
 Repository(full_name="Mirantis/k8s-netchecker-server"),
 Repository(full_name="Mirantis/openstack-lbaas"),
 Repository(full_name="Mirantis/k8s-apps"),
 Repository(full_name="Mirantis/criproxy"),
 Repository(full_name="Mirantis/disk_perf_test_tool"),
 Repository(full_name="Mirantis/ceph-lcm"),
 Repository(full_name="Mirantis/fuel"),
 Repository(full_name="Mirantis/reclass-system-salt-model"),
 Repository(full_name="Mirantis/mk-pipelines"),
 Repository(full_name="Mirantis/launchpad"),
 Repository(full_name="Mirantis/fuelweb"),
 Repository(full_name="Mirantis/k8s-daemonupgradecontroller"),
 Repository(full_name="Mirantis/k8s-devbox"),
 Repository(full

In [1]:
from src.repoextractor import RepoExtractor

RepoExtractor("../data/processed").extract_repo_with_fullname("Mirantis/cri-dockerd")

/Users/nicolegros/Documents/school/release-eng/replication-project/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Extracting commits from repo 'cri-dockerd'
Saving commits to ../data/processed/mirantis/cri-dockerd.commits.pickle


[Commit(sha="9f487a598c226e4cd522b945e1b70af9184fd7f6"),
 Commit(sha="a4152906fcf5ee528d596fcadb9a118019f2f792"),
 Commit(sha="877dc6a4cccd8cf6939878f665db547b86e13767"),
 Commit(sha="b25776320a61d7704396ec58fbb4a650aa817e1d"),
 Commit(sha="c5f285797f217bd034d764240bb176d8c4691f41"),
 Commit(sha="cd730ff87f0ab2bcac1a5172286265917a7907b1"),
 Commit(sha="a6fd175918c8206f0c05e80dd7d51beed6e00a40"),
 Commit(sha="cca7dadc252e34f37457e2168630ff31075cc659"),
 Commit(sha="8c694f1fbeec21a9cee0318beef29f7f2d8ac03f"),
 Commit(sha="cfbffa73011d4aacbedd74cd3cc455f41f170ba0"),
 Commit(sha="7cdfd878214d85155390941a79abf86fb4a06d7a"),
 Commit(sha="665bb2d187845b782cc159c0317bce360214c5a4"),
 Commit(sha="5085bf4e837291e97c7d9ad96d94be9bd05c8029"),
 Commit(sha="d9273a8b458ae5e3291fd1f8bb78415c5b2b5da7"),
 Commit(sha="597d40614901d3635ba196a2394a1c00c0acd2c7"),
 Commit(sha="bec2feb25314acc239a52ef7f8e381cef81bfa1e"),
 Commit(sha="6b452f908a055b89c94988c4b96d2cea2e536e82"),
 Commit(sha="949603f2b55023dc7c

In [13]:
from datetime import datetime
import pandas as pd

dfpick = pd.read_pickle(f"../data/processed/mirantis/cri-dockerd.commits.pickle")


,files,date
9f487a598c226e4cd522b945e1b70af9184fd7f6,"[go.mod, go.sum, vendor/github.com/golang/prot...",2023-10-20
a4152906fcf5ee528d596fcadb9a118019f2f792,"[Makefile, docs/content/about/motivation.md, d...",2023-10-20
877dc6a4cccd8cf6939878f665db547b86e13767,"[Makefile, VERSION, cmd/version/version.go, pa...",2023-10-18
b25776320a61d7704396ec58fbb4a650aa817e1d,[Makefile],2023-10-18
c5f285797f217bd034d764240bb176d8c4691f41,[Makefile],2023-10-18
...,...,...
2ff78e8b5c86e507360a0233505d5e4d6d25cdbd,[dockershim/docker_image.go],2016-08-02
e5f85b3b484dc84c65f807234b1c67fdcb3d68d3,[dockershim/docker_service.go],2016-08-01
ae353c2818e7211fce6d92f6be3528cf807e5b0a,[dockershim/legacy.go],2016-07-29
b6a92c5748286f6edbb486ac866fd1c2bb9f0c02,"[dockershim/docker_container.go, dockershim/do...",2016-07-29


In [2]:
from src.repofinder import RepoFinder

first = RepoFinder.load_repos_from_json('../data/processed/mirantis/repos.json')[0]

/Users/nicolegros/Documents/school/release-eng/replication-project/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
from src.repovalidator import RepoValidator
from datetime import datetime, timedelta

created_at = first["created_at"]
validator = RepoValidator('../data/processed/mirantis/cri-dockerd.commits.pickle')

In [8]:
validator.is_valid()

['.github/ISSUE_TEMPLATE.md', '.github/PULL_REQUEST_TEMPLATE.md', '.github/dependabot.yml', '.github/workflows/build.yml', '.github/workflows/ci.yml', '.github/workflows/e2e.yml', '.github/workflows/hugo.yml', '.github/workflows/integration.yml', '.github/workflows/local-up.yml', '.github/workflows/main.yml', '.github/workflows/publish.yml', '.gitignore', 'LICENSE', 'Makefile', 'README.md', 'VERSION', 'backend/daemon.go', 'backend/daemon_unix.go', 'backend/daemon_windows.go', 'cmd/cri/options/options.go', 'cmd/server.go', 'cmd/version/version.go', 'code-of-conduct.md', 'config/constants.go', 'config/hairpin.go', 'config/options.go', 'config/os.go', 'config/types.go', 'containermanager/container_manager.go', 'containermanager/container_manager_linux.go', 'containermanager/container_manager_unsupported.go', 'containermanager/container_manager_windows.go', 'containermanager/helpers.go', 'containermanager/helpers_linux.go', 'containermanager/helpers_windows.go', 'core/checkpoint.go', 'core

False

In [13]:
import numpy as np
import pandas as pd

validator.df.loc[:, 'month'] = pd.to_datetime(validator.df.date).dt.month
validator.df.loc[:, 'year'] = pd.to_datetime(validator.df.date).dt.year
(validator.df.groupby(['year', 'month']).count() >= 2).all()["files"]



False

In [6]:
list_folder = []
for l in validator.df.files.to_list():
    list_folder.extend(l)
unique_folders = np.unique(list_folder).tolist()

In [7]:
from repofinder import RepoFinder

is_iac_list = []
for folder in unique_folders:
    is_iac_list.append(RepoFinder._is_iac(folder))

In [11]:
print(unique_folders)

['.github/ISSUE_TEMPLATE.md', '.github/PULL_REQUEST_TEMPLATE.md', '.github/dependabot.yml', '.github/workflows/build.yml', '.github/workflows/ci.yml', '.github/workflows/e2e.yml', '.github/workflows/hugo.yml', '.github/workflows/integration.yml', '.github/workflows/local-up.yml', '.github/workflows/main.yml', '.github/workflows/publish.yml', '.gitignore', 'LICENSE', 'Makefile', 'README.md', 'VERSION', 'backend/daemon.go', 'backend/daemon_unix.go', 'backend/daemon_windows.go', 'cmd/cri/options/options.go', 'cmd/server.go', 'cmd/version/version.go', 'code-of-conduct.md', 'config/constants.go', 'config/hairpin.go', 'config/options.go', 'config/os.go', 'config/types.go', 'containermanager/container_manager.go', 'containermanager/container_manager_linux.go', 'containermanager/container_manager_unsupported.go', 'containermanager/container_manager_windows.go', 'containermanager/helpers.go', 'containermanager/helpers_linux.go', 'containermanager/helpers_windows.go', 'core/checkpoint.go', 'core

In [5]:
import os
from repovalidator import RepoValidator
valid_repos = []
for subdir, dirs, files in os.walk("../data/processed/mirantis"):
    for file in files:
        complete_path = os.path.join(subdir, file)
        repo_name = file.split(".")[0]
        if "commits" in complete_path and RepoValidator(complete_path).is_valid():
            valid_repos.append(repo_name)
valid_repos            

[]

In [ ]:
with open("../data/processed/mirantis/validrepos.json", "w") as file:
    json.dump(valid_repos, file)